In [ ]:
# Configure the parameters of the main script
'''The main script is HippoProcess/run.py, it includes:
Hippocampal Segmentation, meshing and registration. You
only need to modifiy lines of 7~9:
(1) The script is customized for batch processing in the server,
    you can set the amount of processes for MRI processing.
(2) Set the inputpath including ‘*nii’ files
(3) Set the outpath for hippocampal mesh files 
'''

In [ ]:
# Configure the batch script to execute the scripts, then excute 'python run.py'
'''HippoProcess/template.sh includes the key parameters to access the server, you only need to
input your email address, if necessary, the server will send you some info about your running.
#!/bin/bash
#SBATCH -n 1                        # number of cores
#SBATCH -t 0-12:00                  # wall time (D-HH:MM)
##SBATCH -A drzuckerman             # Account hours will be pulled from (commented out with double # in front)
#SBATCH -o slurm.%j.out             # STDOUT (%j = JobId)
#SBATCH -e slurm.%j.err             # STDERR (%j = JobId)
#SBATCH --mail-user=qdong17@asu.edu # send-to address'''

In [ ]:
'''[Optional] If you want to learn about the process of your work, you can load the link: 
https://rcstatus.asu.edu/saguaro/jobs.php?jobtype=running'''

In [ ]:
# Inspect the hippocampus mesh results
'''(1) Firstly generate snapshots of left and right hippocampus, you can drag any mesh file of the above results to 'G3dOGL',
       and adjust the suitable angle and use command '>' to get one *.s3d file, rename it as 'rh.s3d' or 'lh.s3d'
   (2) 'run_snap.pl' is the main batch script for snapshot, its content is as following, it will call 'snapshot_new.bat' to 
        generate figs
   (3) By observing snapshots of the generated mesh files, you can reject data with bad quality'''

#set directory of input and output folder
$outputfolder="C:\\snapshot\\snapshot\\outfigs";
$inputfolder="C:\\snapshot\\snapshot\\outfiles_rb";
#$s2dfiledirectory="C:\\snapshot\\snapshot\\l.s3d";
@files = glob "$inputfolder\\*.m";
foreach(@files)
{
    chomp;
    @p = split /\./, $_;
    @q = split /\\/, $_;
        $s2dfiledirectory = (index($q[-1], 'LH') != -1)? "C:\\snapshot\\snapshot\\l.s3d" : "C:\\snapshot\\snapshot\\r.s3d";
    $cmd = "snapshot_new.bat " . $p[0] . " $outputfolder\\" . $q[-1] . " $s2dfiledirectory";
    system($cmd);
};

In [ ]:
# Logitudinal analysis of mesh files, the following script is to make differences betweeen Scan1 and Scan2
import xlrd,glob, os
import subprocess
do_copy = True
sc = ['SCAN_1','SCAN_2']
hs = ['L','R']
exe_path = 'O:\\s1_s2\\SurfStat.exe'
if do_copy:
    
    file_location = "O:\\New_HP_final\\0-all.xls"
    #fn_eval = 'O:\\s1_s2\\eval.txt'
    workbook = xlrd.open_workbook(file_location)
    sheet = workbook.sheet_by_name('Sheet1')
    x = []
    for cell in sheet.col(0):
        for h in hs:
            FNULL = open(os.devnull, 'w') 
            path_m1 = glob.glob('O:\\Data\\HP\\Final_HP\\%s\\%s\\%s*' %(sc[0], h, cell.value))[0]
            path_m2 = glob.glob('O:\\Data\\HP\\Final_HP\\%s\\%s\\%s*' %(sc[1], h, cell.value))[0]
            output_m = 'O:\\Data\\HP\\Final_HP\\SCAN1-2\\%s\\%s_%s_s1-2.m' %(h,cell.value, h)
            args = exe_path + ' -TwoTDiff %s %s %s' %(path_m1, path_m2, output_m)
            #with open(fn_eval, "a") as f:
             #   f.write('%s\n' %args)
            subprocess.call(args, stdout=FNULL, stderr=FNULL, shell=False)

In [ ]:
# Make group comaprisons between genotype groups, you need to prepare several '*。xls' files
# which include group info of each subject，than we can make comparisons using the following command.
python /scratch/qdong17/HippoProcess/HippoUCF/Rainier/UCFGroupDifference/callMMADMTBMGrpDiff.py /scratch/qdong17/Final_HP/exp3/0-1.xls /scratch/qdong17/Final_HP/SCAN1-2/R/ /scratch/qdong17/Final_HP/result/r/s1-2_0-1/ 20000 0.05 grp0 grp1 MADMTBM


# Then you can snapshot the p-maps of different comparisons, using 'G3dOGL.exe'